In [1]:
import os
os.chdir("../")
%pwd

'c:\\Users\\Public\\Documents\\Project\\Document_Based_LLM_chatbot'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
from src.constants import *
from src.utils.common import  read_yaml,create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.csv_source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [5]:
import os
import urllib.request as request
import zipfile
from src import logger
from src.utils.common import get_size 

In [6]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  



In [7]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
except Exception as e:
    raise e

[2024-04-27 22:14:24,647: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-27 22:14:24,649: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-27 22:14:24,651: INFO: common: created directory at: artifacts]
[2024-04-27 22:14:24,652: INFO: common: created directory at: artifacts/data_ingestion]
[2024-04-27 22:14:24,654: INFO: 2515186901: File already exists of size: ~ 320 KB]
